In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
.appName('Caching in Dataframes').enableHiveSupport().getOrCreate()

25/06/19 07:09:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Ran this Earlier lets, do working on the large file

In [3]:
custumers_df = spark.read.option('header','true').csv('/data/customers_100.csv')

In [4]:
custumers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: string (nullable = true)
 |-- is_active: string (nullable = true)



In [7]:
custumers_df.cache()

DataFrame[customer_id: string, name: string, city: string, state: string, country: string, registration_date: string, is_active: string]

In [8]:
tail_df = custumers_df.orderBy('customer_id',ascending = False)

In [9]:
tail_df.show()

+-----------+-----------+---------+-----------+-------+-----------------+---------+
|customer_id|       name|     city|      state|country|registration_date|is_active|
+-----------+-----------+---------+-----------+-------+-----------------+---------+
|         98|Customer_98|     Pune|West Bengal|  India|       2023-06-01|     True|
|         97|Customer_97|     Pune|Maharashtra|  India|       2023-10-31|     True|
|         96|Customer_96|Ahmedabad|  Telangana|  India|       2023-06-22|    False|
|         95|Customer_95|   Mumbai|  Karnataka|  India|       2023-09-29|     True|
|         94|Customer_94|  Chennai|Maharashtra|  India|       2023-04-28|    False|
|         93|Customer_93|     Pune|West Bengal|  India|       2023-09-30|    False|
|         92|Customer_92|  Kolkata|      Delhi|  India|       2023-06-25|     True|
|         91|Customer_91|   Mumbai|  Telangana|  India|       2023-01-02|    False|
|         90|Customer_90|Ahmedabad|    Gujarat|  India|       2023-02-15|   

In [10]:
custumers_df.unpersist()

DataFrame[customer_id: string, name: string, city: string, state: string, country: string, registration_date: string, is_active: string]

caching on a bigger file

### handling bigger data

In [3]:
!hadoop fs -ls /data/

Found 5 items
-rw-r--r--   2 root               hadoop       5488 2025-06-02 15:26 /data/customers_100.csv
-rw-r--r--   2 geurecruitanandraj hadoop   10528211 2025-06-17 15:58 /data/customers_10mb.csv
-rw-r--r--   2 geurecruitanandraj hadoop  168541068 2025-06-17 15:56 /data/customers_150mb.csv
-rw-r--r--   2 geurecruitanandraj hadoop    8628599 2025-06-17 15:58 /data/orders_10.csv
-rw-r--r--   2 geurecruitanandraj hadoop    7896035 2025-06-17 15:59 /data/shippings_10.csv


In [4]:
spark

In [5]:
cust_df = spark.read.option('header','true').csv('/data/customers_150mb.csv')

In [6]:
cust_df.rdd.getNumPartitions()

2

In [7]:
cust_df.show(5)

+-----------+----------+---------+---------+-------+-----------------+---------+
|customer_id|      name|     city|    state|country|registration_date|is_active|
+-----------+----------+---------+---------+-------+-----------------+---------+
|          0|Customer_0|  Kolkata|Karnataka|  India|       2023-12-21|    False|
|          1|Customer_1|Ahmedabad|Telangana|  India|       2023-03-07|     True|
|          2|Customer_2|Bangalore|Karnataka|  India|       2023-01-14|    False|
|          3|Customer_3|  Chennai|Karnataka|  India|       2023-11-16|    False|
|          4|Customer_4|    Delhi|Telangana|  India|       2023-05-23|     True|
+-----------+----------+---------+---------+-------+-----------------+---------+
only showing top 5 rows



In [8]:
cust_df.cache()

DataFrame[customer_id: string, name: string, city: string, state: string, country: string, registration_date: string, is_active: string]

### tail() is a big operation , let's see wheather this tail utilizes the cache memory or not ?


In [10]:
cust_df.tail(2)

[Row(customer_id='2612729', name='Customer_2612729', city='Ahmedabad', state='Delhi', country='India', registration_date='2023-10-13', is_active='True'),
 Row(customer_id='2612730', name='Customer_2612730', city='Pune', state='West Bengal', country='India', registration_date='2023-12-29', is_active='True')]

In [12]:
tail_df = cust_df.orderBy('customer_id',ascending = False)
tail_df.show(5)

+-----------+---------------+---------+-----------+-------+-----------------+---------+
|customer_id|           name|     city|      state|country|registration_date|is_active|
+-----------+---------------+---------+-----------+-------+-----------------+---------+
|     999999|Customer_999999|Bangalore|  Karnataka|  India|       2023-10-22|     True|
|     999998|Customer_999998|   Mumbai|    Gujarat|  India|       2023-07-14|     True|
|     999997|Customer_999997|Hyderabad|    Gujarat|  India|       2023-07-26|    False|
|     999996|Customer_999996|Hyderabad|  Telangana|  India|       2023-03-17|    False|
|     999995|Customer_999995|  Chennai|Maharashtra|  India|       2023-08-23|    False|
+-----------+---------------+---------+-----------+-------+-----------------+---------+
only showing top 5 rows



### even if we cache the data for multiple partitons , then depending on our previous actions , spark would cache data for only one partition , as we did the above operation , caching would not help as the data cached is for only single partition.But if we do the action without caching , first time it would take time and then give us the results , this time it would cache the whole partition data , so if we run thr code for the second time , it would be faster as it has cached the whole data.

## Second use case

In [23]:
cust_df.schema

StructType([StructField('customer_id', StringType(), True), StructField('name', StringType(), True), StructField('city', StringType(), True), StructField('state', StringType(), True), StructField('country', StringType(), True), StructField('registration_date', StringType(), True), StructField('is_active', StringType(), True)])

In [18]:
# what should happen in case of this , how much partitions should be cached in following case if we had 5 partitions
df_filtered_10 = cust_df.filter((cust_df.customer_id >='10000')&(cust_df.customer_id <='400000'))

In [19]:
df_filtered_10.show(5)

+-----------+-----------+---------+----------+-------+-----------------+---------+
|customer_id|       name|     city|     state|country|registration_date|is_active|
+-----------+-----------+---------+----------+-------+-----------------+---------+
|          2| Customer_2|Bangalore| Karnataka|  India|       2023-01-14|    False|
|          3| Customer_3|  Chennai| Karnataka|  India|       2023-11-16|    False|
|          4| Customer_4|    Delhi| Telangana|  India|       2023-05-23|     True|
|         11|Customer_11|     Pune|     Delhi|  India|       2023-02-22|     True|
|         12|Customer_12|  Kolkata|Tamil Nadu|  India|       2023-06-25|     True|
+-----------+-----------+---------+----------+-------+-----------------+---------+
only showing top 5 rows



In [20]:
df_filtered.count()

1946066

### Question here is how many partitions would heve been cached , by default we would think only 2 or 3 , but filter is a wide transformation so it would cache whole data as the filtered data could be in any partition.

In [24]:
spark.stop()